# Trading algorithm that “works”

https://trading-data-analysis.pro/trading-algorithm-that-doesnt-work-37e747f4c6a6

Approach based on 3 daily bars predicted by neural network and technical indicators such as Bollinger Bands and RSI.

#### Prerequisites

- Trading API from Alpaca Markets (https://alpaca.markets). Set up your balance to 29k.
- In order to notify us about bot’s execution we will need a Channel. I usually use Telegram-channel and bot cause they are the easiest to setup and monitor. Get HTTPS Keys.
- Telegram Channel, with the bot as Administrator. Get channel ID.

#### Tools and libraries

- Python
- 
Google Collaboration (Colab)- 
alpaca_trade_ap- i
yfinan- ce
pandas- _ta
requ- ests
matplotlib.

#### Imports

We have to import the following list of libraries which we will be using during the preparation of our trading script.
  pyplot

In [ ]:
%pip install -r requirements1.txt

In [21]:
import requests as rq
import alpaca_trade_api as api
import yfinance as yf
import pandas_ta as ta
import telebot

#### Settings

For our trading algorithm we will use some settings. Here’s the short explanation of each of them:
- Parameter ‘TRADER_BOT_NAME’ is the bot name which we will be using in notifications to the Telegram-channel;
- Parameters ‘TRADER_API_KEY’, ‘TRADER_API_SECRET’ and ‘TRADER_API_URL’ should be filled with appropriate values which you received from the Trader API at Alpaca dashboard;
- Parameters ‘TELEGRAM_URL’, ‘TELEGRAM_BOT_ID’ and ‘TELEGRAM_CHAT_ID’ should be filled with appropriate values which we discussed above;
- Parameter ‘SCREENER_INTERVAL’ setup to 5m means that we will use 5 minutes chart for the stock to analyze it;
- Parameter ‘SCREENER_PERIOD’ setup to 250m means that we will check the window for 250 bars;
- Parameter ‘SCREENER_NASDAQ_COUNT’ that we have set up to 500 means that our stock screener will screen just only first 500 stocks out of more than 2300 shortable stocks in NASDAQ index;
- Parameter ‘TAKE_PROFIT_DELTA’ that we have set up to 0.02 means that we will calculate and let algorithm trade when projected profit is at least 2% from the current stock price;
- Parameter ‘CASH_LIMIT’ is the minimal amount on trading account which we don’t want to trade. We set it up to 26k because with the total balance 29k it means that just only 3k will be used in trading.

In [38]:
# SETTINGS
TRADER_BOT_NAME = 'Trading_AI_Bot'

TRADER_API_KEY = 'PKUH3V70OHPE6CWU3F4L'
TRADER_API_SECRET = 'cswvwZ2CeLiYchIBQcwA4UUEy7Ph81BCsjoQ3lCe'
TRADER_API_URL = 'https://paper-api.alpaca.markets'

TELEGRAM_URL = 'https://api.telegram.org'
TELEGRAM_BOT_ID = '6240899521:AAEjX6ayWtTb1FSU-OIZ8lbCoX7szwA3_jk'
TELEGRAM_CHAT_ID = '-4010532996'

SCREENER_INTERVAL = '5m'
SCREENER_PERIOD = '250m'
SCREENER_NASDAQ_COUNT = 500

TAKE_PROFIT_DELTA = 0.02
CASH_LIMIT = 26000

#### Send messages
In order to know how everything is going and what is our trading balance, we use the notification function the declaration of which is listed below:

In [39]:
# Send message to Telegram channel
def send_message(message):
  response = rq.post(
        f'{TELEGRAM_URL}/{TELEGRAM_BOT_ID}/sendMessage?chat_id={TELEGRAM_CHAT_ID}&parse_mode=Markdown&text={message}')

  return response

In [23]:
#Codigo alternativo para mandar mensajes al Bot

import telebot

bot = telebot.TeleBot("6240899521:AAEjX6ayWtTb1FSU-OIZ8lbCoX7szwA3_jk")
@bot.message_handler(commands=["help","start"])

def enviar(message):
    bot.reply_to(message, "Hola, ¿Cómo estás?")

bot.polling()

In [40]:
send_message("prueba")

<Response [404]>

As I mentioned earlier, all messages will be sent to the Telegram-channel using regular HttpRequest. Here is just one parameter in query string which I would like to explain: it’s a ‘parse_mode’ which is set up to ‘Markdown’ that means the messages will be formatted.

#### Stock screener

Awesome! So we are now at the most important section of this article and hereby I will explain why. I believe that Stock-screener is the crucial tool of any algorithm because it lets us preselect the stocks which are more relevant to our preconditions and worth our attention at this moment.

As you remember from the ‘Settings’ chapter we will screen the first 500 stocks from the NASDAQ index to find the pattern which we believe is the most relevant for tradiIn our case, to identify this pattern we use Bollinger Bands with period = 20 and non-standard deviation equals to 2.3. This number is important, believe me. I got this setup for BBANDS based on my own experiments. They work much better than the standard deviation equaling 2. Actually these numbers are so important that most investment firms protect them in their public relations and non-disclosure agreements.

And the second indicator is RSI with standard setup.ng.



In [ ]:
# Check stock with TA indicators
def CheckStock(stock):
  data = {}
  try:
    df = yf.download(stock, period = SCREENER_PERIOD, interval = SCREENER_INTERVAL)
    if (len(df) > 0):
      df['RSI'] = ta.rsi(df['Close'], timeperiod=14)
      bbands = ta.bbands(df['Close'], length = 20, std=2.3)
      df['L'] = bbands['BBL_20_2.3']
      df['M'] = bbands['BBM_20_2.3']
      df['U'] = bbands['BBU_20_2.3']
      
      previous2_bar = df[-3:].head(1)
      previous_bar = df[-2:].head(1)
      current_bar = df[-1:]

      if current_bar['RSI'].values[0] > 70 and \
          current_bar['Close'].values[0] > current_bar['U'].values[0]:
            data = { 'direction': 'DOWN', 'stock' : stock, 'stop_loss': round(max(previous_bar['High'].values[0], previous2_bar['High'].values[0], previous_bar['U'].values[0]), 2), \
                    'take_profit': round(min(previous_bar['Low'].values[0], previous2_bar['Low'].values[0], previous_bar['M'].values[0]), 2) }
      elif current_bar['RSI'].values[0] < 30 and \
            current_bar['Close'].values[0] < current_bar['L'].values[0]:
              data = { 'direction': 'UP', 'stock' : stock, 'stop_loss': round(min(previous_bar['Low'].values[0], previous2_bar['Low'].values[0], previous_bar['L'].values[0]), 2), \
                      'take_profit': round(max(previous_bar['High'].values[0], previous2_bar['High'].values[0], previous_bar['M'].values[0]), 2) }
  except:
    pass

  return data

In [ ]:
#Chequeamos el valor de algunas acciones
CheckStock("QCOM")

Let me explain how it works. We load tickers for first 500 stocks from NASDAQ and check each of that stock at this particular moment for several conditions.

For the market that is gonna go DOWN (market is overbought):

- Current stock price should be above the upper band of BBANDS;
- Current RSI value is above 70.

For the market that is gonna go UP (market is oversold):

- Current stock price should be below the lower band of BBANDS;
- Current RSI value is below 30.

In [ ]:
# Screen stocks
def ScreenStocks(trader_api):
  assets = trader_api.list_assets(status='active', asset_class='us_equity')
  assets = [x for x in assets if x.shortable == True and x.exchange == 'NASDAQ']
  stocks = [x.symbol for x in assets][:SCREENER_NASDAQ_COUNT]

  screened = []
  for st in stocks:
    _stock = CheckStock(st)
    if _stock != {}:
      screened.append(_stock)

  screened = [x for x in screened if abs(x['stop_loss'] - x['take_profit']) > min(x['stop_loss'], x['take_profit']) * TAKE_PROFIT_DELTA]
  return screened

At the same time, during this stock prescreen we prepare (calculate) our stop-loss and take-profit in advance and add the prescreened stock to the collection of stocks selected. I know, you probably wonder ‘Where exactly we use the neural network and its predictions?’. Be patient, you will see it next.

#### Prediction from neural network

When you prescreened the stock you might want to increase your chances to win the market using the predictions from the neural network.

In the previous article we discussed how to predict stock market with Tensorflow and LSTM neural network. As you probably remember, by the end of that script, we got 3 numbers — predictions from neural network for the upcoming 3 days.

In [ ]:
# Get predictions from LSTM neural network
def Predict(stock):
  predictions = [100.24, 155.33, 140.55]

  # Here we have to organize communication between our algorithm and LSTM Model \
  # to get predictions by ticker for the particular stock.
  # But this is the question to the infrastructure.
  # I am gonna consider it in the next article "Infrastructure itself".

  return predictions

The task of this section is using those predictions. However, this is little complicated thing. Remember, our LSTM neural network works hard, at least 10 minutes for data mining the result and generating predictions. It means that we need to build the appropriate infrastructure to let it work asynchronously.

For instance, in one of my own working algorithms I use database to store and read predictions from. Let’s discuss our ways to communicate between trading algorithms and neural networks next time.

But, for purposes of this experiment we will return some list of numbers as if they were our predictions from the procedure ‘Predict(stock)’.

#### Let’s trade
The function ‘Trade’ is the basic function which creates all orders on trading platform. It has a few input parameters:

- Parameter ‘api’ is the reference to API-object;
- Parameter ‘stock’ is the string with an appropriate ticker which we will buy or sell;
- Parameter ‘operation’ could have the value ‘buy’ or ‘sell’;
- Parameter ‘shares_to_trade’ is the quantity of shares which we will buy;
- Parameter ‘take_profit’ is the target price for exit from the market;
- Parameter ‘stop_loss’ is the insurance from the situation when something goes wrong and market goes in the opposite direction from what we have set up to follow;

In [ ]:
# Trade with STOP_LOSS and TAKE_PROFIT
def Trade(api, stock, operation, shares_to_trade, take_profit, stop_loss):
  api.submit_order(symbol = stock, qty = shares_to_trade, side = operation, type = 'market',
                  order_class = 'bracket', time_in_force = 'day', 
                  take_profit = {'limit_price': take_profit},
                  stop_loss = {'stop_price': stop_loss})
  message = f'\n\t*{stock}*, qty _{shares_to_trade}_ \n\t\twere {operation}'
  send_message(f'{TRADER_BOT_NAME}: we entered the market with:' + message)
  return True

When the order is placed we send the message to the appropriate Telegram-channel to notify us about the trading.

#### Trading Bot

Ok, we are almost at the last section of this article and now we are going to consider this final part of the code. This script describes the logic of our algorithm from the beginning to the end.

I want to emphasize that this script is expected to be launched each hour from 9:30 AM until 3:30 PM from Monday to Friday except holidays. At this time the market is open.

At the very beginning we open the connection with Trader API using our API_KEY and API_SECRET which we set up in the chapter ‘Settings’. Then we also load information about our account and market clock.

First of all we check if account was loaded and if so, we send the message to Telegram-channel about our total balance and non-marginable buying power which we can use to trade. This is the amount of money which is available for trading now.

In [ ]:
# MAIN script
def medium_trader_go(request):
  trader_api = api.REST(TRADER_API_KEY, TRADER_API_SECRET, TRADER_API_URL)
  account = trader_api.get_account()
  clock = trader_api.get_clock()

  if bool(account) == True:
    message = f'''{TRADER_BOT_NAME}: for *{account.account_number}*
    current capital is _{account.portfolio_value}$_ 
    and non marginable buying power is _{account.non_marginable_buying_power}$_'''
    send_message(message)

  if clock.is_open == True:
    if float(account.non_marginable_buying_power) < CASH_LIMIT:
      message = f"{TRADER_BOT_NAME}: there is no cash on the account or limit reached!"
      send_message(message)
    else:
      # Screen stocks
      screened = ScreenStocks(trader_api)
      # Check limit and trade
      if len(screened) > 0:
        CASH_FOR_TRADE_PER_SHARE = (float(account.non_marginable_buying_power) - CASH_LIMIT) / len(screened)
        for item in screened:
          predictions = Predict(item['stock'])
          STOCK = item['stock']
          OPERATION = 'buy' if item['direction'] == 'UP' else 'sell'
          STOP_LOSS = min([item['stop_loss']] + predictions) if item['direction'] == 'UP' else max([item['stop_loss']] + predictions)
          TAKE_PROFIT = max([item['take_profit']] + predictions) if item['direction'] == 'UP' else min([item['take_profit']] + predictions)
          SHARE_PRICE = round(min(STOP_LOSS, TAKE_PROFIT), 2)
          SHARES_TO_TRADE = int(CASH_FOR_TRADE_PER_SHARE / SHARE_PRICE)
          try:
            if abs(STOP_LOSS - TAKE_PROFIT) > SHARE_PRICE * TAKE_PROFIT_DELTA and SHARES_TO_TRADE > 0:
              Trade(api, STOCK, OPERATION, SHARES_TO_TRADE, TAKE_PROFIT, STOP_LOSS)
              print(f'\n{STOCK}: {STOP_LOSS}, {TAKE_PROFIT}, {OPERATION}, {SHARES_TO_TRADE}')
          except:
            pass

  portfolio = trader_api.list_positions()
  if bool(portfolio) == True:
    message = f'{TRADER_BOT_NAME}: we have {len(portfolio)} opened positions.'
    for i in portfolio:
      message = message + f'\n\t*{i.symbol}*: qty {i.qty} {i.side} for _{i.market_value}$_ \n\t\t\tcurrent price _{i.current_price}$_ \n\t\t\tprofit _{i.unrealized_pl}$_'
    send_message(message)
  
  if clock.is_open == False:
    message = f"{TRADER_BOT_NAME}: the market is *CLOSED*, let's try later on!"
    send_message(message)

  return f'{TRADER_BOT_NAME}: DONE!'

We also check if the market is open and if we have enough money to trade (more than CASH_LIMIT). Then we screen the market as it was described in the chapter ‘Stock screener’, get the prediction for each stock from neural network and start the trading. On each step we notify Telegram-channel about the work done.

Finally, we list all the stocks from our portfolio to the channel. From this information we can analyze how the trading is going.

#### Bot’s work

Because of the fact that we have our Telegram-channel setup we do not need to open the dashboard to see how our algorithm works, if it is profitable or it has lost some money — we can see all of those things simply in the Telegram-chan
I have removed the name of one of my own experimental channels to keep it secure. However when your own bot will be working you will see a very similar listing in your iPhone as well as I can see it in here.

#### This is not the end of the story

As usually, in the end you probably expect to read the conclusion and that the story is finished and you can go and eat those soft French croissants. However, don’t rush. I highly recommend you to double check all these concepts and scripts on paper account (not real) with virtual money and ensure that this concept works particularly in your case. The same concept can work for one trader and not work for another one. It depends on many factors, even on the amount of money you have as the deposit on your trading account. And this is not a joke. Do you know that FINRA prohibits intraday trades for customers that don’t have at least 25k on their accounts?

I want you to remember: profit and success on the stock market depends on many factors. The trading itself is risky and any strategy should be double checked before being applied to the real trading account.

In this and also in the previous article you have got everything you need to start experimenting with these or any other trading strategy. Use them ‘for good’ and don’t forget to double check any strategy before applying them to the market itself. And good lu#### ck :)

To be continued
The next time I will describe another important thing in automated trading, — the environment for the components of your tradi (https://trading-data-analysis.pro/where-to-host-your-trading-bot-692e51369912)ng scripts, cause I don’t think you wish to launch those scripts manually :)

As we use Tensorflow for the predictions from Google and Python as a programming language I would recommend you to use Google Cloud Functions and Scheduler because of simplicity and small costs.

